In [1]:
import sys; sys.path.insert(0, "..")

import aoc

year, day = 2020, 20

puzzle = aoc.setup(year, day)
plines = puzzle.splitlines()

# Day 20

### Puzzle 1

In [2]:
import math


def edge_to_int(edge):
    x1 = int("".join(edge).replace(".", "0").replace("#", "1"), 2)
    x2 = int("".join(reversed(edge)).replace(".", "0").replace("#", "1"), 2)
    return min(x1, x2)
    
def solve1():
    m = {}
    for num, *lines in map(str.splitlines, puzzle.split("\n\n")):
        t = list(zip(*lines))
        num = int(num.split()[1][:-1])
        for e in map(edge_to_int, [lines[0], lines[-1], t[0], t[-1]]):
            m.setdefault(e, []).append(num)
    border = [v[0] for k, v in m.items() if len(v) == 1]
    return math.prod(set([x for x in border if border.count(x) == 2]))

solve1()

15670959891893

In [3]:
solve1m=lambda E=lambda e:min(int(x:="".join(e).translate({46:48,35:49}),2),int(x[::-1],2)):(m:={})or[m.setdefault(e,[]).append(int(n.split()[1][:-1]))for n,*L in map(str.splitlines,puzzle.split("\n\n"))if(t:=list(zip(*L)))for e in map(E,[L[0],L[-1],t[0],t[-1]])]and(b:=[v for v,*x in m.values()if not x])and __import__("math").prod({x for x in b if b.count(x)==2})

solve1m()

15670959891893

In [4]:
%timeit solve1()
%timeit solve1m()

2.47 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.1 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Puzzle 2

In [5]:
op = [1, 0, 3, 2]
monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.splitlines()

def rotflip(tile, o):
    c = (3, 1)
    k = True
    while c != o:
        if k:
            tile = list(zip(*tile))
            c = c[::-1]
        else:
            tile = [line[::-1] for line in tile]
            c = (op[c[0]], c[1])
        k = not k
    return list(map("".join, tile))


def solve2():
    m = {}
    tiles = {}
    for num, *lines in map(str.splitlines, puzzle.split("\n\n")):
        t = list(zip(*lines))
        num = int(num.split()[1][:-1])
        tiles[num] = [line[1:-1] for line in lines[1:-1]]
        for i, e in enumerate(map(edge_to_int, [lines[0], lines[-1], t[0], t[-1]])):
            m.setdefault(e, []).append((i, num))
    
    G = {}
    for (o1, t1), (o2, t2) in [v for k, v in m.items() if len(v) == 2]:
        G.setdefault(t1, {})[o1] = (o2, t2)
        G.setdefault(t2, {})[o2] = (o1, t1)
    
    corner = next(k for k, v in G.items() if len(v) == 2)

    border = {corner}
    dim = []
    for k in G[corner]:
        c = corner
        x = 1
        while k in G[c]:
            k, c = G[c][k]
            k = op[k]
            border.add(c)
            x += 1
        dim.append(x)
    width, height = dim
    
    for k, v in G.items():
        if k in border:
            continue
        for i in range(4):
            v.setdefault(i, None)
    
    tiled_image = [[None] * width for _ in range(height)]
    
    queue = [(corner, 0, 0, *G[corner])]
    visited = set()
    while queue:
        p, i, j, a, b = queue.pop(0)
        
        if p in visited: continue
        visited.add(p)
        
        tiled_image[i][j] = rotflip(tiles[p], (a, b))
        
        for o1, u, v in [(a, i, j+1), (b, i+1, j)]:
            if G[p].get(o1) is None:
                continue
            o2, q = G[p][o1]
            if len(G[q]) >= 3:
                G[q].pop(o2)

            if len(G[q]) <= 2:
                if o1 == a:
                    x = op[o2]
                    y, = set(G[q]) - {x, o2}
                else:
                    y = op[o2]
                    x, = set(G[q]) - {y, o2}
                queue.append((q, u, v, x, y))

    image = []
    for tile_line in tiled_image:
        image += ["".join(line) for line in zip(*tile_line)]
    
    monster_parts = set()
    for i in range(4):
        for j in range(4):
            if i in (j, op[j]):
                continue
            m = rotflip(monster, (i, j))
            for y in range(len(image) - len(m) + 1):
                for x in range(len(image[0]) - len(m[0]) + 1):
                    p = [(y+i, x+j) for i, line in enumerate(m) for j, c in enumerate(line) if c == "#"]
                    if all(image[i][j] == "#" for i, j in p):
                        monster_parts.update(p)
    return sum(line.count("#") for line in image) - len(monster_parts)


solve2()

1964

In [6]:
solve2m=lambda z=enumerate,Y=range,O=[1,0,3,2],E=lambda e:min(int(x:="".join(e).translate({46:48,35:49}),2),int(x[::-1],2)):(R:=lambda t,o,c=(3,1):c==o and[*map("".join,t)]or[3,2,0,1][c[0]]==c[1]and R([*zip(*t)],o,c[::-1])or R([l[::-1]for l in t],o,(O[c[0]],c[1])),m:={},P:={},[m.setdefault(e,[]).append((i,n))for N,*l in map(str.splitlines,puzzle.split("\n\n"))if(t:=list(zip(*l)),n:=int(N.split()[1][:-1]),P.update({n:[L[1:-1]for L in l[1:-1]]}))for i,e in z(map(E,[l[0],l[-1],t[0],t[-1]]))],G:={},[(G.setdefault(t1,{}).update({o1:(o2,t2)}),G.setdefault(t2,{}).update({o2:(o1,t1)}))for(o1,t1),(o2,t2)in[v for k,v in m.items()if len(v)==2]],B:={(C:=next(k for k,v in G.items()if len(v)==2))},D:=[(U:=(lambda k,c:B.add(c)or(1+U(O[G[c][k][0]],G[c][k][1])if k in G[c]else 1)))(k,C)for k in G[C]],[v.setdefault(i,None)for k, v in G.items()if k not in B for i in Y(4)],T:=[[None]*D[0]for _ in Y(D[1])],Q:=[(C,0,0,*G[C])],V:=set(),next(0 for()in iter(tuple,0)if not Q or(p:=Q.pop(0),p[0]not in V and(V.add(p[0]),T[p[1]].__setitem__(p[2],R(P[p[0]],p[3:5])),[(3<=len(G[f[1]])and G[f[1]].pop(f[0]),2>=len(G[f[1]])and(A==p[3]and(x:=O[f[0]],y:=[*(set(G[f[1]])-{x,f[0]})][0])or(y:=O[f[0]],x:=[*(set(G[f[1]])-{y,f[0]})][0]),Q.append((f[1],u,v,x,y))))for A,u,v in[(p[3],p[1],p[2]+1),(p[4],p[1]+1,p[2])]if(f:=G[p[0]].get(A))]))*0),I:=["".join(l)for t in T for l in zip(*t)])and sum(l.count("#")for l in I)-len({s for i in Y(4)for j in Y(4)if i not in(j,O[j])and(m:=R([18*" "+"# ",3*"#    #"+"##",6*" # "+"  "],(i,j)))for y in Y(len(I)-len(m)+1)for x in Y(len(I[0])-len(m[0])+1)if(p:=[(y+i,x+j)for i,l in z(m)for j,c in z(l)if"#"==c])if all(I[i][j]=="#"for i,j in p)for s in p})

solve2m()

1964

In [7]:
%timeit solve2()
%timeit solve2m()

643 ms ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
613 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
